# F1 Race Simulation Notebook

This notebook test the `Run` class and demonstrates how to simulate races on the 2016 F1 timing database with both deterministic and stochastic modes.

## Configuration

In [6]:
import os
import sys

In [7]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Project root added to sys.path:", project_root)

Project root added to sys.path: c:\Users\mateo\Desktop\MCEM\new\Nouveau dossier\Simulation_Project


In [8]:
# === General configuration ===
season = 2016
location="Austin"
db_path = "../data/F1_timingdata_2014_2019.sqlite"

# === List of test GPs with associated strategy files for a given season ===

gp_configs = {
    "Austin": {
        "strategy_file": "strategies_austin_2016.pkl",
        "var_name": "simulator_austin"
    },
    "Suzuka": {
        "strategy_file": "strategies_suzuka_2016.pkl",
        "var_name": "simulator_suzuka"
    },
    "MexicoCity": {
        "strategy_file": "strategies_mexico_2016.pkl",
        "var_name": "simulator_mexico"
    },
    "YasMarina": {
        "strategy_file": "strategies_yasmarina_2016.pkl",
        "var_name": "simulator_yasmarina"
    }
}


## Load and Preprocess Data

In [9]:
from data_loader import DataLoader
from preprocessor import DataPreprocessor

# Load raw data
loader = DataLoader(db_path)
raw_data = loader.load_data()

required_columns = {
    "drivers": [
        "id",
        "name",
        "initials",
    ],
    "races": [
        "id",
        "season",
        "location",
        "nolapsplanned",
    ],
    "starterfields": [
        "race_id",
        "driver_id",
        "team",
        "status",
        "resultposition",
    ],
    "laps": [
        "race_id",
        "driver_id",
        "lapno",
        "laptime",
        "racetime",
        "compound",
        "tireage",
        "pitintime",
        "pitstopduration",
    ],
    "qualifyings": [
        "race_id",
        "driver_id",
        "position",
        "q1laptime",
        "q2laptime",
        "q3laptime",
    ],
    "retirements": [
        "season",
        "driver_id",
        "accidents",
        "failures",
    ],
    "fcyphases": [
        "race_id",
        "startlap",
        "endlap",
    ],
}



preprocessor = DataPreprocessor(required_columns)
data_tables = preprocessor.preprocess(raw_data)


## Run Stochastic Simulations

In [5]:
import pickle
from run import Run

# === Dictionary to store simulators by name ===
Probabilistic_simulators = {}

# === Loop through each GP ===
for gp, cfg in gp_configs.items():
    print("=" * 60)
    print(f" Running Simulation for: {gp}")

    # Load strategy
    with open(f"../data/{cfg['strategy_file']}", "rb") as f:
        driver_strategies = pickle.load(f)

    # Create simulator
    sim = Run(season, 
              gp, 
              data_tables, 
              driver_strategies)

    sim.run()

    # Store simulator with dynamic name (and in dict)
    globals()[cfg["var_name"]] = sim.outcomes
    Probabilistic_simulators[cfg["var_name"]] = sim.outcomes
    print(f"Simulation completed for: {gp}")
    print("=" * 60)


 Running Simulation for: Austin


KeyboardInterrupt: 

## Run Deterministic Simulation

In [10]:
races = data_tables["races"]
race = races[
    (races["season"] == 2016)
    & (races["location"] == "MexicoCity")
]
race_id = int(race["id"].iloc[0])
print(race_id)

57


In [12]:
data_tables["drivers"]

,id,name,initials
0,1,Lewis Hamilton,HAM
1,2,Daniel Ricciardo,RIC
2,3,Nico Rosberg,ROS
3,4,Kevin Magnussen,MAG
4,5,Fernando Alonso,ALO
5,6,Jean-Eric Vergne,VER
6,7,Nico Hulkenberg,HUL
7,8,Daniil Kvyat,KVY
8,9,Felipe Massa,MAS
9,10,Jenson Button,BUT


In [ ]:
data_tables["qualifyings"].loc[(data_tables["qualifyings"]["driver_id"]==30)
                               &
                               (data_tables["qualifyings"]["race_id"]==57)]

,race_id,driver_id,position,q1laptime,q2laptime,q3laptime
620,39,30,14,87.241,87.601,NaN
648,40,30,20,93.438,NaN,NaN
669,41,30,19,99.528,NaN,NaN
690,42,30,18,99.009,NaN,NaN
711,43,30,17,84.903,NaN,NaN
733,45,30,17,75.459,NaN,NaN
759,46,30,22,106.394,NaN,NaN
776,49,30,17,103.965,NaN,NaN
797,50,30,16,76.636,76.665,NaN
817,51,30,14,108.901,108.888,NaN


In [ ]:
data_tables["drivers"].loc[data_tables["drivers"]["id"]==30].iloc[0][["name", "initials"]]

name        Jolyon Palmer
initials              PAL
Name: 29, dtype: object

In [ ]:
data_tables["starterfields"]

,race_id,driver_id,team,status,resultposition
0,3,3,Mercedes,F,2
1,3,1,Mercedes,F,1
2,3,15,Williams,F,8
3,3,16,ForceIndia,F,3
4,3,11,Ferrari,F,10
...,...,...,...,...,...
2115,121,35,AlfaRomeo,F,16
2116,121,11,AlfaRomeo,F,13
2117,121,44,Williams,F,17
2118,121,45,Williams,F,19


In [ ]:
data_tables["laps"].loc[(data_tables["laps"]["driver_id"]==30)
                               &
                               (data_tables["laps"]["race_id"]==57)]

,race_id,driver_id,lapno,laptime,racetime,compound,tireage,pitintime,pitstopduration
51143,57,30,0,NaN,0.000,A4,0,None,NaN
51163,57,30,1,120.672,120.672,A4,1,2016-10-30 13:05:06,23.897
51185,57,30,2,117.697,238.369,A2,1,None,NaN
51206,57,30,3,111.065,349.434,A2,2,None,NaN
51227,57,30,4,87.724,437.158,A2,3,None,NaN
...,...,...,...,...,...,...,...,...,...
52523,57,30,66,88.237,5745.794,A2,65,None,NaN
52544,57,30,67,86.260,5832.054,A2,66,None,NaN
52565,57,30,68,85.257,5917.311,A2,67,None,NaN
52586,57,30,69,85.038,6002.349,A2,68,None,NaN


In [ ]:
import pickle
from run import Run

# === Dictionary to store simulators by name ===
simulators = {}

# === Loop through each GP ===
for gp, cfg in gp_configs.items():
    print("=" * 60)
    print(f" Running Simulation for: {gp}")
    

    # Load strategy
    with open(f"../data/{cfg['strategy_file']}", "rb") as f:
        driver_strategies = pickle.load(f)

    # Create simulator
    sim = Run(season, 
              gp, 
              data_tables, 
              driver_strategies,
              test_mode=True)

    sim.run()

    # Store simulator with dynamic name (and in dict)
    globals()[cfg["var_name"]] = sim.outcomes
    simulators[cfg["var_name"]] = sim.outcomes
    print(f"Simulation completed for: {gp}")
    print("=" * 60)


 Running Simulation for: Austin


[INFO] Race simulation completed.


Simulation completed for: Austin
 Running Simulation for: Suzuka


[INFO] Race simulation completed.


Simulation completed for: Suzuka
 Running Simulation for: MexicoCity


[INFO] Race simulation completed.


Simulation completed for: MexicoCity
 Running Simulation for: YasMarina


[INFO] Race simulation completed.


Simulation completed for: YasMarina


## VIsualisation of the outputs